## What is HuggingFace?
Hugging Face also provide many open source models, and a platform to host the models.

In [5]:
# Required libraries
! pip install sentence_transformers langchain_huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
# Connect to LLM using my libraries
from utility.llm_factory import LLMFactory

embedding_model = LLMFactory.get_embedding_model("huggingface", model_name="all-MiniLM-L6-v2")
embedding_model

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Sentence Transformers on Hugging Face
Hugging Face sentence-transformers is a python framework for state-of-the-art sentence, text and image embeddings. One of the embedding models is used in the HuggingFaceEmbeddings class. We have also added an alias for SentenceTransformerEmbeddings for users who are more famillar with directly using that package.

In [7]:

r1= embedding_model.embed_query("Why is the sky blue?")
r2= embedding_model.embed_documents(["The sky is blue because of the way sunlight interacts with the Earth's atmosphere."])
print(r1)
print(r2)


[0.010076949372887611, -0.0017390934517607093, 0.05018017441034317, 0.04702519625425339, 0.05492459610104561, 0.00869524572044611, 0.10545976459980011, -0.025911128148436546, 0.12947818636894226, 0.03196641802787781, -0.04449514299631119, -0.008958651684224606, -0.0005428381846286356, -0.06376104056835175, -0.016147548332810402, 0.04666970670223236, -0.022032422944903374, -0.1582026332616806, -0.07292037457227707, -0.061305586248636246, -0.06590540707111359, 0.054199669510126114, -0.06216304749250412, 0.03894707188010216, -0.04587395489215851, 0.054934173822402954, -0.03527248278260231, 0.01266008522361517, 0.04252506047487259, -0.007990885525941849, -0.019042594358325005, 0.06094198301434517, 0.0369039811193943, 0.01350767444819212, -0.025806181132793427, -0.043561674654483795, 0.07244455069303513, -0.04851972684264183, 0.004293750040233135, -0.029454175382852554, -0.029107267037034035, -0.03296322748064995, -0.018351396545767784, 0.015543718822300434, -0.01179700531065464, 0.01526409

In [8]:
## Step2: Load a text file as a document
from langchain_community.document_loaders import TextLoader

loader = TextLoader('./_data/speech.txt', encoding='utf-8')
docs = loader.load()

docs

[Document(metadata={'source': './_data/speech.txt'}, page_content="Good morning everyone,\n\nToday, I want to talk about something incredibly simple, yet profoundly powerful: small steps.\n\nIn a world obsessed with big wins and overnight success, we often forget that every great achievement starts with a single small action.\n\nWhether you're trying to learn a new skill, change a habit, or build something meaningful — it always begins with the decision to take one small step forward.\n\nThink about the tallest buildings. They're built one brick at a time. Olympic athletes? They train for years, often making tiny improvements day after day.\n\nSo, the next time you feel overwhelmed by your goals, just focus on the next step. Not the next ten, not the whole staircase — just the next one.\n\nProgress isn’t always loud. Sometimes, it whispers.\n\nBut those whispers? They build momentum.\n\nAnd that momentum? It builds success.\n\nThank you.\n\n")]

In [9]:
## Step 3: Split the document into smaller chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter


with open('./_data/speech.txt', 'r', encoding='utf-8') as f:
    speech = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=20
)

#final_documents = text_splitter.split_text(speech)

# Note: the input here is a list of text strings, not Document objects.
document_chunks = text_splitter.create_documents([speech])
document_chunks[::]  # Display the first two chunks

[Document(metadata={}, page_content='Good morning everyone,'),
 Document(metadata={}, page_content='Today, I want to talk about something incredibly simple, yet profoundly'),
 Document(metadata={}, page_content='yet profoundly powerful: small steps.'),
 Document(metadata={}, page_content='In a world obsessed with big wins and overnight success, we often forget that'),
 Document(metadata={}, page_content='often forget that every great achievement starts with a single small action.'),
 Document(metadata={}, page_content="Whether you're trying to learn a new skill, change a habit, or build something"),
 Document(metadata={}, page_content='or build something meaningful — it always begins with the decision to take one'),
 Document(metadata={}, page_content='to take one small step forward.'),
 Document(metadata={}, page_content="Think about the tallest buildings. They're built one brick at a time. Olympic"),
 Document(metadata={}, page_content='at a time. Olympic athletes? They train for yea

#### FAISS (Facebook AI Simillarity Search)
FAISS is a library for efficient simillarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tunning.

In [10]:
## Step4: Create embeddings for the document chunks
## and store them in a vector database
from langchain.vectorstores import FAISS #(Using faiss-cpu for testing)
vector_store_db = FAISS.from_documents(document_chunks, embedding_model)

vector_store_db

In [11]:
## Step5: Query the vector store
query = "Progress isn’t always loud. Sometimes, it whispers."
results = vector_store_db.similarity_search(query, k=3)
print(type(results))
# Display the results
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(result.page_content)
    print("-" * 40)

<class 'list'>
Result 1:
Progress isn’t always loud. Sometimes, it whispers.
----------------------------------------
Result 2:
But those whispers? They build momentum.
----------------------------------------
Result 3:
And that momentum? It builds success.

Thank you.
----------------------------------------


In [12]:
# Step6: store the db to local folder
vector_store_db.save_local("./_data/faiss_db")



In [13]:
# Step7: Load the vector store from local folder
from langchain.vectorstores import FAISS
vector_store_db_loaded = FAISS.load_local("./_data/faiss_db", 
embedding_model, 
allow_dangerous_deserialization=True)


In [14]:
## Step8: Query the vector store
query = "Progress isn’t always loud. Sometimes, it whispers."
results = vector_store_db_loaded.similarity_search(query, k=3)
print(type(results))
# Display the results
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(result.page_content)
    print("-" * 40)

<class 'list'>
Result 1:
Progress isn’t always loud. Sometimes, it whispers.
----------------------------------------
Result 2:
But those whispers? They build momentum.
----------------------------------------
Result 3:
And that momentum? It builds success.

Thank you.
----------------------------------------
